In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image, ImageDraw
from IPython import display
from lib.viz import showarray


import os, time, json, io
#os.environ["MLFLOW_TRACKING_URI"] = "sqlite:///mlruns/db.sqlite"
#import mlflow
#mlflow_client = mlflow.tracking.MlflowClient()

from lib import eos

#from lib.eos import EyeOnStickEnv
#ENV = EyeOnStickEnv

from lib.pyb.eos3d import EyeOnStickEnv3D

from lib.run import run_env_nsteps

import logging
logger = logging.getLogger()

In [ ]:
from lib.run import mk_env_agent
from lib.dashboard import get_episode_dashboard

In [ ]:
env_class = EyeOnStickEnv3D
model_name='eos3d.6j-coarse-aim'
model_version = None
params = {'NJ': 6, 'NP': 4, 'MAX_NSTEPS': 150} # 'GEAR_FUNC_NOISE': 0, 
gui = False

lastrun_metrics = {}
lastrun_trajs = None
lastrun_gearfuncs = None

while True:
    env, model = mk_env_agent(env_class, model_name, params, gui=gui)
    
    # run the episode, collect images of rendered environment and trajectory data
    # FIXME: era_env_render = np.zeros((params['MAX_NSTEPS'], eos.SCREEN_SIZE[0], eos.SCREEN_SIZE[1], 3))
    lastrun_env_images = None
    def displayfunc(img_array):
        global lastrun_env_images
        
        img_array = np.hstack((img_array, env.envs[0].render_step_dashboard()))
        img_array = img_array[np.newaxis,...] # make it (1, SCREEN_SIZE-X, SCREEN_SIZE-y, 3)
        if lastrun_env_images is None:
            lastrun_env_images = img_array
        else:
            lastrun_env_images = np.vstack((lastrun_env_images, img_array))
        # (?, SCREEN_SIZE-X, SCREEN_SIZE-y)

    lastrun_trajs = None
    def trajfunc(traj): # (3, NJ) where 3 corresponds to phi and _phi and dphi
        global lastrun_trajs
        traj = traj[np.newaxis,:,:] # make it (1, 2, NJ)
        if lastrun_trajs is None:
            lastrun_trajs = traj
        else:
            lastrun_trajs = np.vstack((lastrun_trajs, traj))
        # (?, 2, NJ)

    logger.debug(f"run_env_nsteps: n_steps={params['MAX_NSTEPS']}")
    lastrun_metrics, data = run_env_nsteps(env, model, params['MAX_NSTEPS'], displayfunc=displayfunc, trajfunc=trajfunc)
    lastrun_gearfuncs = list(env.envs[0].gearfuncs) # FIXME
    
    episode_dashboard_img_array = get_episode_dashboard(lastrun_gearfuncs, lastrun_trajs, lastrun_metrics)
        
    for i in range(params['MAX_NSTEPS']):
        display.clear_output(wait=True)

        #img_array = np.hstack(( # , episode_dashboard_img_array))
        img_array = np.vstack((lastrun_env_images[i], episode_dashboard_img_array))
        showarray(img_array)
        time.sleep(0.1)
    
    env.close()